출처 : [pytorch 튜토리얼](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)  
  
이 포스트는 머신러닝의 일반적인 작업에 대한 pytorch api를 살펴본다.

## Working with data
pytorch에는 데이터를 다루는데 2가지 기본 요소가 있다: `torch.utils.data.DataLoader`와 `torch.utils.data.Dataset`. `Dataset`은 샘플과 그에 대응되는 레이블을 저장하고, `DataLoader`는 `Dataset`의 python iterable 객체이다.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

pytorch는 [TorchText](https://pytorch.org/text/stable/index.html), [TorchVision](https://pytorch.org/vision/stable/index.html), [TorchAudio](https://pytorch.org/audio/stable/index.html)와 같은 도메인에 특화한 라이브러리를 제공하고 각각 데이터셋을 포함한다. 이 튜토리얼에서는 TorchVision 데이터셋을 사용한다.  
  
`torchvision.datasets` 모듈은 CIFAR, COCO ([모든 리스트](https://pytorch.org/vision/stable/datasets.html))와 같은 실제 vision 데이터의 `Datasets` 객체를 포함하고 있다. 이 튜토리얼에서는 FashionMNIST dataset을 사용한다. 모든 TorchVision `Dataset`은 두 가지 arguments를 포함한다: 샘플과 레이블 각각 수정하기 위한 `transform`과 `target_transform`.

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

`DataLoader`의 argument로 `Dataset`을 전달한다. `DataLoader`는 dataset의 iterable을 감싸고, 자동 배치, 샘플링, 셔플링 및 멀티프로세싱을 지원한다. 배치 사이즈를 64로 지정하면 dataloader는 64개의 피쳐와 레이블 배치를 리턴한다.

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models
pytorch에서 뉴럴 네트워크를 정의하려면 `nn.Module`을 상속하는 class를 만들어야한다. `__init__` function에서 네트워크의 레이어를 정의하고 `forward` function에서 데이터가 어떻게 통과할 지 정의한다. 연산을 가속화하기 위해 GPU를 사용할 수 있다.  
자세한 내용은 [building neural networks in PyTorch](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html) 참고.

In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters
모델을 학습하기 위해 [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions)과 [optimizer](https://pytorch.org/docs/stable/optim.html)가 필요하다.

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

한번의 학습 루프에서 모델은 학습 dataset에 대해 predictions를 만들고, 모델의 parameters를 조정하기 위해 prediction error를 backpropagate를 한다.

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

학습이 잘 되고 있는지 test dataset에 대해 모델의 성능 측정도 해야한다.

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

학습은 여러번 실행된다. 각 epoch마다 모델은 더 나은 predictions를 위해 parameters를 조정한다. epoch 마다 accuracy의 증가와 loss의 감소를 볼 수 있다.

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

## Saving Models
모델을 저장하는 방법 중 하나는 내부 상태 dictionary를 직렬화하는 것이다. 이 dictionary에는 모델 paramters를 포함하고 있다.

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models
모델을 load하려면 모델의 구조를 다시 만들고 상태 dictionary를 load한다.  
더 자세한 내용은 [Saving & Loading your model](https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html) 참고.

In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
